In [20]:
import numpy as np
import operator
import logging
import time

import matplotlib.pyplot as plt

import optuna

import jax.numpy as jnp
from jax import grad, jit

import optax
from optax import adam 

from functools import partial
import gwjax
import gwjax.imrphenom

from pycbc.catalog import Merger
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.waveform import get_fd_waveform
from pycbc.filter import matched_filter, sigmasq, get_cutoff_indices

In [21]:
class NeedsInvestigatingCallback(object):
    """A callback for Optuna which identifies potential events."""

    def __init__(self, early_stopping_rounds: int, snr_threshold: int, direction: str = "minimize") -> None:
        self.snr_threshold = snr_threshold
        self.early_stopping_rounds = early_stopping_rounds
        
        self._iter = 0

        if direction == "minimize":
            self._operator = operator.lt
            self._score = np.inf
        elif direction == "maximize":
            self._operator = operator.gt
            self._score = -np.inf
        else:
            ValueError(f"invalid direction: {direction}")

    def __call__(self, study: optuna.Study, trial: optuna.Trial) -> None:
        """Goes onto Stocastic Gradient Descent."""
        if self._operator(study.best_value, self._score):
            self._iter = 0
            self._score = study.best_value
        else:
            self._iter += 1

        if self._score <= self.snr_threshold:
            if self._iter >= self.early_stopping_rounds:
                study.stop()
                print(study.best_params)
                
        if self._score >= self.snr_threshold:
            study.stop()
            print(f'Optuna has stopped with {study.best_params} because snr threshold has been reached.')
            # Perform gradient descent to find the lowest SNR
            initial_m1 = study.best_params['m1'] #from the Optuna section of GWtuna
            initial_m2 = study.best_params['m2'] #from the Optuna section of GWtuna
            initial_s1 = 0.0
            initial_s2 = 0.0
            print(f'Jax is about to start using Gradient Descent')
            optimized_m1, optimized_m2, optimized_s1, optimized_s2 = gradient_descent(initial_m1, initial_m2, initial_s1, initial_s2, my_waveform_template, my_matched_filter, my_snr)
            print("Optimized Mass and Spin Parameters: m1 =", optimized_m1, "m2 =", optimized_m2, "s1 =", optimized_s1, "s2 =", optimized_s2)
            print("The SNR is with the optimized parameters:", -objective([optimized_m1, optimized_m2, optimized_s1, optimized_s2]))

In [22]:
# Define the conditiion data function
dynfac = 1.0e23
def condition(strain, sampling_rate):
    tmp = resample_to_delta_t(highpass(strain, 15.0), 1.0/sampling_rate).crop(2,2)
    return (dynfac*tmp)

# Define the PSD function 
def estimate_psd(strain, delta_f):
    psd = strain.psd(4)
    psd = interpolate(psd, delta_f)
    psd = inverse_spectrum_truncation(psd, int(4 * strain.sample_rate),
                                  low_frequency_cutoff=15, trunc_method='hann')
    return psd

# Define the matched filter function
def matched_filter(signal_duration, sampling_rate, kmin, kmax, fcore, template): 
    workspace = jnp.zeros(int(sampling_rate*signal_duration), dtype=complex) 
    result_fft = fcore * jnp.conj(template) #jnp.conjugate(pycbc_template)
    workspace = workspace.at[kmin:kmax].set(result_fft)
    result = jnp.fft.ifft(workspace)
    result *= len(result)
    return result

# Define the waveform template function
def waveform_template(freqs, m1, m2, s1, s2):
    params = {'phase': 0., 'geocent_time': 0.,
            'luminosity_distance': 1, 'theta_jn': 0., 
            'm1': m1, 'm2': m2, 'spin1': s1, 'spin2': s2,
            'ra': 0., 'dec': 0., 'pol': 0.}
    # Create the waveform template using the mass parameters
    # Adjust this function based on your specific waveform model
    #may want to do partial on this
    return gwjax.imrphenom.IMRPhenomD(freqs, params)

# Define sigma squared function 
def sigma_squared(delta_freq, invpsd, template):
    weighted_inner = jnp.sum(template*jnp.conj(template)*invpsd)
    h_norm = 4*delta_freq
    sigma_squared = jnp.real(weighted_inner)*h_norm
    #print(f'sigma_squared {sigma_squared}')
    return sigma_squared

# Define the waveform template function
def optuna_waveform_template(freqs, m1, m2):
    params = {'phase': 0., 'geocent_time': 0.,
            'luminosity_distance': 1, 'theta_jn': 0., 
            'm1': m1, 'm2': m2, 'spin1': 0, 'spin2': 0,
            'ra': 0., 'dec': 0., 'pol': 0.}
    # Create the waveform template using the mass parameters
    # Adjust this function based on your specific waveform model
    #may want to do partial on this
    return gwjax.imrphenom.IMRPhenomD(freqs, params)

def snr(invpsd, delta_freq, sampling_rate, matched_output, sigma_squared_output):
    norm = 4*delta_freq / jnp.sqrt(sigma_squared_output)
    snr = matched_output*norm
    snr_min = int((2+4)*sampling_rate)
    snr_max = len(snr)-int((2)*sampling_rate)
    snr = snr[snr_min:snr_max]
    peak = jnp.argmax(jnp.absolute(snr))
    snrp = jnp.absolute(snr[peak])
    return(snrp)

# Define the objective function to minimize (negative SNR)
def objective(params):
    m1, m2, s1, s2 = params
    template, _ = my_waveform_template(m1, m2, s1, s2)
    #print(template)
    matched_output = my_matched_filter(template)
    #print(matched_output)
    sigma_squared_output = my_sigma_squared(template)
    #print(sigma_squared_output)
    snr_peak = my_snr(matched_output, sigma_squared_output)
    #print(snr_peak)
    return -snr_peak

# Perform gradient descent
def gradient_descent(initial_m1, initial_m2, initial_s1, initial_s2, my_waveform_template, my_matched_filter, my_snr):
    precision = 0.001
    iters = 0 
    Total_gradient = 0.1
    
    # Set the Optimizer
    adam = optax.adam(learning_rate=1.0)
    
    # Initialize the parameters
    m1, m2, s1, s2 = initial_m1, initial_m2, initial_s1, initial_s2
    
    # Define the gradient of the objective function
    objective_grad = jit(grad(objective))
    opt_fn=adam.update
    opt_state=adam.init(objective)

    while abs(float(Total_gradient)) > precision:
        
        # Compute the gradient
        grad_m1, grad_m2, grad_s1, grad_s2 = objective_grad((m1, m2, s1, s2))
        # Update the parameters
        m1_updates, opt_state = opt_fn(grad_m1, opt_state)
        m2_updates, opt_state = opt_fn(grad_m2, opt_state)
        s1_updates, opt_state = opt_fn(grad_s1, opt_state)
        s2_updates, opt_state = opt_fn(grad_s2, opt_state)
        m1 += m1_updates
        m2 += m2_updates
        s1 += s1_updates
        s2 += s2_updates
        
        iters = iters+1 #iteration count
        
        Total_gradient = abs(float(grad_m1))
        
        if iters%100 == 0:
            print("Iteration",iters,"\n values is", m1, m2, s1, s2, "\n Total Gradient is", abs(float(grad_m1)))
        
        if s1<-0.99 or s1>0.99:
            print('Help the spin is out of range')
            break
            
    return m1, m2, s1, s2

In [23]:
# Get the data and estimate the PSDs
merger = Merger("GW150914")
sampling_rate = 2048 # Sampling rate in Hz
data = condition(merger.strain('H1'), sampling_rate)
signal_duration = float(data.duration)  # Duration of the signal in seconds
delta_freq = data.delta_f

invpsd = estimate_psd(data, data.delta_f)**(-1)

fcore = data.to_frequencyseries()*invpsd

#Get the frequency range
nyquist = sampling_rate//2
freqs = jnp.arange(1+(nyquist*signal_duration))/signal_duration


#Determine the low freq and high frequency cut off
kmin, kmax = int(15*signal_duration), int(900*signal_duration)

#Make everything the same length
fcore = jnp.asarray(fcore[kmin:kmax])
freqs = freqs[kmin:kmax]
invpsd = jnp.asarray(invpsd[kmin:kmax]) # can't take fft because it needs to start at 0 to nyquist 

#Get the stage ready for the template
my_waveform_template = partial(waveform_template, freqs)
my_optuna_waveform_template = partial(optuna_waveform_template, freqs)
my_matched_filter = partial(matched_filter, signal_duration, sampling_rate, kmin, kmax, fcore)
my_sigma_squared = partial(sigma_squared, delta_freq, invpsd)
my_snr = partial(snr, invpsd, delta_freq, sampling_rate)

In [24]:
def optuna_objective(trial):
    m1 = trial.suggest_float('m1', 2, 100, step=0.000001)
    m2 = trial.suggest_float('m2', 2, 100, step=0.000001)
    template, _ = my_optuna_waveform_template(m1, m2)
    matched_output = my_matched_filter(template)
    sigma_squared_output = my_sigma_squared(template)
    snr_peak = my_snr(matched_output, sigma_squared_output)
    return snr_peak

In [25]:
start_time = time.time()
optuna.logging.disable_default_handler()
direction="maximize"
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction=direction)
needs_to_be_investigated = NeedsInvestigatingCallback(300, snr_threshold=19, direction=direction)
study.optimize(optuna_objective, callbacks=[needs_to_be_investigated], n_trials=500)
print("Time taken", time.time() - start_time)
print(f'GW150419 has the best {study.best_params} with a snr {study.best_value}')

Optuna has stopped with {'m1': 57.287161, 'm2': 21.418568999999998} because snr threshold has been reached.
Jax is about to start using Gradient Descent


TypeError: zeros_like requires ndarray or scalar arguments, got <class 'function'> at position 0.